In [1]:
# add absolute path to the project root folder to the system path
import sys
import os
from pathlib import Path

notebook_path = Path().resolve()
sys.path.append(str(notebook_path.parent))
import warnings
from scipy.stats import trim_mean
from scipy.optimize import minimize
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from utils.timescale_connector import TimescaleConnector

In [2]:
# Preparing financial ratios data
def financial_ratio():
    _df = TimescaleConnector.query_financial_ratios()

    _columns = [
        "quarter",
        "net_profit",
        "profit_growth_(%)",
        "revenue",
        "revenue_growth_(%)",
        "market_capital",
        "eps_(vnd)",
        "p/e",
        "outstanding_share",
        "roe_(%)",
        "symbol",
    ]

    _df = _df[_columns]

    _trimmed_mean_roe = trim_mean(_df["roe_(%)"].dropna(), 0.1)
    _trimmed_mean_market_capital = trim_mean(_df["market_capital"].dropna(), 0.1)
    _trimmed_mean_eps = trim_mean(_df["eps_(vnd)"].dropna(), 0.1)
    _trimmed_mean_pe = trim_mean(_df["p/e"].dropna(), 0.1)
    _trimmed_mean_outstanding_share = trim_mean(_df["outstanding_share"].dropna(), 0.1)

    _df["roe_(%)"].fillna(_trimmed_mean_roe, inplace=True)
    _df["market_capital"].fillna(_trimmed_mean_market_capital, inplace=True)
    _df["eps_(vnd)"].fillna(_trimmed_mean_eps, inplace=True)
    _df["p/e"].fillna(_trimmed_mean_pe, inplace=True)
    _df["outstanding_share"].fillna(_trimmed_mean_outstanding_share, inplace=True)

    _revenue_null_value = _df[_df["revenue"].isnull()]
    _revenue_null_value.head(5)

    _financial_ratios_cleaned = _df.dropna()
    return _financial_ratios_cleaned


df = financial_ratio()


In [3]:
df


,quarter,net_profit,profit_growth_(%),revenue,revenue_growth_(%),market_capital,eps_(vnd),p/e,outstanding_share,roe_(%),symbol
0,Q4 2016,41294860102,0.949968,6.676867e+11,0.451578,1.455455e+12,817.936153,10.250735,56964988.0,0.160569,AAA
1,Q1 2017,66894667573,1.358928,7.335012e+11,0.728754,1.949325e+12,1303.709320,9.762348,59249988.0,0.195754,AAA
2,Q2 2017,39129314217,0.162533,8.538257e+11,0.650835,1.970062e+12,733.182009,9.805468,59249988.0,0.193482,AAA
3,Q3 2017,52029043164,0.345329,1.062929e+12,0.981942,2.399320e+12,948.578466,8.722134,83599988.0,0.200697,AAA
4,Q4 2017,65289636792,0.581060,1.426508e+12,1.136493,3.344000e+12,1117.613576,7.486253,167199976.0,0.202759,AAA
...,...,...,...,...,...,...,...,...,...,...,...
2067,Q3 2022,220666896949,6.246926,6.521616e+11,1.262953,8.362940e+12,931.039774,7.810071,236241246.0,0.245687,VSH
2068,Q4 2022,382126600956,0.982613,9.628110e+11,0.518069,9.520522e+12,1612.272023,7.551561,236241246.0,0.273152,VSH
2069,Q1 2023,476648446226,0.180250,8.921769e+11,0.103405,1.058361e+13,2011.079450,7.944936,236241246.0,0.272947,VSH
2070,Q2 2023,261808202789,0.018630,6.572797e+11,-0.005774,1.060723e+13,1104.623545,7.908467,236241246.0,0.265446,VSH


In [4]:
def analyze_industry_sector():
    _df_industry_sector = pd.read_csv(
        "/home/tb24/projects/rule-based-stock-recommendation-system/data/industry_sector_analysis.csv",
        encoding="ISO-8859-1",
    )
    _df_industry_sector = _df_industry_sector.dropna()
    # rename industry sector to industry_sector
    _df_industry_sector.rename(
        columns={"industry sector": "industry_sector"}, inplace=True
    )

    _corrections = {
        "Hóa Ch?t": "Hóa Chất",
        "Hóa ch?t": "Hóa Chất",
        "B?t ??ng s?n": "Bất Động Sản",
        "S?n xu?t th?c ph?m": "Sản xuất thực phẩm",
        "D?ch v? tài chính": "Dịch vụ tài chính",
        "Xây d?ng và v?t li?u": "Xây dựng và vật liệu",
        "B?o hi?m nhân th?": "Bảo hiểm nhân thọ",
        "N??c và khí ??t": "Nước và khí đốt",
        "Ph?n m?m d?ch v? máy tính": "Phần mềm dịch vụ máy tính",
        "Bán l?": "Bán lẻ",
        "Lâm nghi?p và gi?y": "Lâm nghiệp và giấy",
        "Ph?n m?m và d?ch v? máy tính": "Phần mềm và dịch vụ máy tính",
        "S?n xu?t và phân ph?i ?i?n": "Sản xuất và phân phối điện",
        "?i?n t? và thi?t b? ?i?n": "Điện tử và thiết bị điện",
        "V?n t?i": "Vận tải",
        "Kim lo?i": "Kim loại",
        "D??c ph?m": "Dược phẩm",
        "S?n xu?t d??c ph?m": "Sản xuất dược phẩm",
        "S?n xu?t d?u khí": "Sản xuất dầu khí",
        "Hàng cá nhân": "Hàng cá nhân",
        "Thi?t b?, d?ch v? và phân ph?i d?u khí": "Thiết bị, dịch vụ và phân phối dầu khí",
        "Công nghi?p n?ng": "Công nghiệp nặng",
        "Bia và ?? u?ng": "Bia và đồ uống",
        "Thi?t b? và ph?n c?ng": "Thiết bị và phần cứng",
        "Du l?ch và gi?i trí": "Du lịch và giải trí",
        "Ch? s? th? tr??ng chung": "Chỉ số thị trường chung",
    }

    _df_industry_sector.industry_sector = _df_industry_sector.industry_sector.replace(
        _corrections
    )
    _df_industry_sector = _df_industry_sector.reset_index()
    _df_industry_sector = _df_industry_sector.drop(["index"], axis=1)

    _df_industry_sector.date = pd.to_datetime(_df_industry_sector.date)
    _df_industry_sector.close = pd.to_numeric(
        _df_industry_sector.close, errors="coerce"
    )
    _df_industry_sector.dropna(subset=["close"], inplace=True)

    _df_vn_index = _df_industry_sector[_df_industry_sector.symbol == "VN-Index"]
    _df_vn_index["vn_index_change"] = _df_vn_index["close"].pct_change()
    _df_industry_sector["stock_price_change"] = _df_industry_sector.groupby("symbol").close.pct_change()

    _df_merged = _df_industry_sector.merge(
        _df_vn_index[["date", "vn_index_change"]], on="date", how="left"
    )

    _grouped = _df_merged.groupby("industry_sector").agg(
        {"stock_price_change": "mean", "vn_index_change": "mean"}
    )

    _grouped["RS"] = _grouped["stock_price_change"] / _grouped["vn_index_change"]
    _ranked_sectors = _grouped.sort_values(by="RS", ascending=False)
    _ranked_sectors.reset_index(inplace=True)
    _ranked_sectors["ranking"] = _ranked_sectors.reset_index(drop=False).index + 1
    return _ranked_sectors

In [5]:
df2 = analyze_industry_sector()
df2

/tmp/ipykernel_63131/1755996557.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df_vn_index["vn_index_change"] = _df_vn_index["close"].pct_change()


,industry_sector,stock_price_change,vn_index_change,RS,ranking
0,Thiết bị và phần cứng,-0.001685,-0.000059,28.795372,1
1,"Thiết bị, dịch vụ và phân phối dầu khí",0.001446,0.000476,3.039476,2
2,Phần mềm dịch vụ máy tính,0.001247,0.000537,2.322306,3
3,Phần mềm và dịch vụ máy tính,0.001304,0.000623,2.092782,4
4,Ngân hàng,0.000694,0.000369,1.882444,5
5,Vận tải,0.000518,0.000280,1.849879,6
6,Nước và khí đốt,-0.000083,-0.000057,1.469318,7
7,Bán lẻ,-0.000346,-0.000253,1.367405,8
8,Chỉ số thị trường chung,0.000119,0.000119,1.000000,9
9,Xây dựng và vật liệu,0.000636,0.000748,0.849964,10


In [6]:
def process_dataframe(_df):
    _df["eps_growth(%)"] = (
        (_df["eps_(vnd)"] - _df.groupby("symbol")["eps_(vnd)"].shift(4))
        / _df.groupby("symbol")["eps_(vnd)"].shift(4)
    ) * 100
    _df = _df.dropna()

    _df["profit_growth_(%)"] *= 100
    _df["revenue_growth_(%)"] *= 100

    _df_fundamental = _df[
        [
            "quarter",
            "net_profit",
            "profit_growth_(%)",
            "revenue",
            "revenue_growth_(%)",
            "eps_(vnd)",
            "eps_growth(%)",
            "roe_(%)",
            "symbol",
        ]
    ]
    return _df_fundamental


df_fundamental_ = process_dataframe(df)


/tmp/ipykernel_63131/3932778355.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["profit_growth_(%)"] *= 100
/tmp/ipykernel_63131/3932778355.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  _df["revenue_growth_(%)"] *= 100


# Fundamental + Techinical Rules

In [7]:
# All the rules:
from dataclasses import dataclass


@dataclass
class FundamentalAnalysis:
    df_fundamental: pd.DataFrame
    latest_year: str
    latest_quarter: str
    recent_2_quarters: list[str]
    recent_3_quarters: list[str]
    growth_threshold_eps: int
    growth_threshold_revenue: int
    growth_threshold_roe: int

    # The most recent quarter's EPS growth is greater than 15% compared to the same quarter of the previous year
    def check_eps_growth_1stcondition(self):
        latest_quarter_df = self.df_fundamental[
            self.df_fundamental["quarter"] == self.latest_quarter
        ]
        latest_quarter_df["condition_met"] = (
            latest_quarter_df["eps_growth(%)"] > self.growth_threshold_eps
        )
        return latest_quarter_df[["symbol", "eps_growth(%)", "condition_met"]]

    # EPS growth for the two most recent quarters is greater than 15% compared to the same quarters of the previous year
    def check_eps_growth_2ndcondition(self):
        recent_quarter_df = self.df_fundamental[
            self.df_fundamental["quarter"].isin(self.recent_2_quarters)
        ]
        result = recent_quarter_df.groupby("symbol").apply(
            lambda x: (x["eps_growth(%)"] > self.growth_threshold_eps).all()
        )
        return result

    # Earnings Per Share (EPS) in each quarter of the last 12 months is at or near its peak
    def assess_eps_near_peak_3rdcondition(self):
        year_df = self.df_fundamental[
            self.df_fundamental["quarter"].str.contains(self.latest_year)
        ]

        def is_eps_at_peak(stock_df):
            max_eps = 0
            for eps in stock_df["eps_(vnd)"]:
                near_peak = max_eps * 0.95
                if eps < near_peak:
                    return False
                max_eps = max(max_eps, eps)
            return True

        result = year_df.groupby("symbol").apply(is_eps_at_peak)
        return result

    # Most recent quarter's revenue is greater than 20% compared to the same quarter of the previous year
    def check_revenue_growth_4thcondition(self):
        latest_quarter_df = self.df_fundamental[
            self.df_fundamental["quarter"] == self.latest_quarter
        ]
        latest_quarter_df["condition_met"] = (
            latest_quarter_df["revenue_growth_(%)"] > self.growth_threshold_revenue
        )
        return latest_quarter_df[["symbol", "revenue_growth_(%)", "condition_met"]]

    # Accelerating revenue growth over the last three quarters
    def check_accelerating_revenue_growth_5thcondition(self):
        filtered_df = self.df_fundamental[
            self.df_fundamental["quarter"].isin(self.recent_3_quarters)
        ]

        def is_growth_accelerating(stock_df):
            stock_df = stock_df.sort_values(by="quarter")
            growth_rates = stock_df["revenue_growth_(%)"].tolist()
            return all(x < y for x, y in zip(growth_rates, growth_rates[1:]))

        result = filtered_df.groupby("symbol").apply(is_growth_accelerating)
        return result

    # Accelerating profit growth over the last three quarters
    def check_accelerating_profit_growth_6thcondition(self):
        filtered_df = self.df_fundamental[
            self.df_fundamental["quarter"].isin(self.recent_3_quarters)
        ]

        def is_growth_accelerating(stock_df):
            stock_df = stock_df.sort_values(by="quarter")
            growth_rates = stock_df["profit_growth_(%)"].tolist()
            return all(x < y for x, y in zip(growth_rates, growth_rates[1:]))

        result = filtered_df.groupby("symbol").apply(is_growth_accelerating)
        return result

    # Return on Equity (ROE) in the most recent quarter is greater than 15 %
    def check_roe_7thcondition(self):
        current_quarter_df = self.df_fundamental[
            self.df_fundamental["quarter"] == self.latest_quarter
        ]
        current_quarter_df["condition_met"] = (
            current_quarter_df["roe_(%)"] >= self.growth_threshold_roe
        )
        return current_quarter_df[["symbol", "roe_(%)", "condition_met"]]


@dataclass
class TechnicalAnalysis:
    df_technical: pd.DataFrame
# fmt: off
# Start of Technical Rules
# Calculate EMA34-89 for each stock
    
    def calculate_ema(self):
        for i in [34, 89]:
            self.df_technical[f"EMA{i}"] = (
                self.df_technical
                .groupby("symbol")["close"]
                .transform(lambda x: x.ewm(span=i, adjust=False).mean())
                .dropna()
            )
        return self.df_technical

    # Calculate MACD for each stock
    def calculate_ma(self):
        for i in [5, 10, 20, 50, 150, 200]:
            self.df_technical[f"MA{i}"] = (
                self.df_technical\
                .groupby("symbol")["close"]
                .transform(lambda x: x.rolling(i).mean())
                .dropna()
            )
        return self.df_technical    
    
    def check_emacross_8thcondition(self):
        self.df_technical["EMA_Condition_Met"] = self.df_technical["EMA34"] > self.df_technical["EMA89"]
        condition_met_by_stock = self.df_technical.groupby("symbol")["EMA_Condition_Met"].any()
        return condition_met_by_stock
    
    def check_ma_9thcondition(self):
        self.df_technical["MA_Condition_Met"] = (
            self.df_technical["MA50"] > self.df_technical["MA150"]
        ) & (self.df_technical["MA150"] > self.df_technical["MA200"])
        condition_met_by_stock = self.df_technical.groupby("symbol")["MA_Condition_Met"].any()
        return condition_met_by_stock

In [8]:
# Get the stock prices for technical analysis
from utils.timescale_connector import TimescaleConnector

df_technical_ = TimescaleConnector.query_ohlcv_all()
df_technical_

,symbol,date,open,high,low,close,volume
0,AAA,2017-01-03 00:00:00,12.50,12.95,12.50,12.81,694080
1,AAA,2017-01-04 00:00:00,13.13,13.23,12.76,12.97,740770
2,AAA,2017-01-05 00:00:00,13.02,13.23,12.42,13.23,1057020
3,AAA,2017-01-06 00:00:00,13.02,13.05,12.50,12.76,823010
4,AAA,2017-01-09 00:00:00,12.76,13.65,12.55,13.54,777440
...,...,...,...,...,...,...,...
163775,VSH,2023-12-28 00:00:00,44.80,46.00,44.50,45.00,12800
163776,VSH,2023-12-29 00:00:00,45.20,45.20,44.50,44.80,12900
163777,VSH,2024-01-02 00:00:00,44.80,45.30,44.60,45.30,14200
163778,VSH,2024-01-03 00:00:00,48.30,48.30,45.60,46.50,14300


In [9]:
# Test the FundamentalRules class

# Init the class
evaluate = FundamentalAnalysis(
    df_fundamental=df_fundamental_,
    latest_year="2023",
    latest_quarter="Q3 2023",
    recent_2_quarters=["Q3 2023", "Q2 2023"],
    recent_3_quarters=["Q1 2023", "Q2 2023", "Q3 2023"],
    growth_threshold_eps=15,
    growth_threshold_revenue=20,
    growth_threshold_roe=15,
)

In [10]:
# Test the first condition
evaluate.check_eps_growth_1stcondition()

/tmp/ipykernel_63131/77979599.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_quarter_df["condition_met"] = (


,symbol,eps_growth(%),condition_met
27,AAA,10.483045,False
83,ANV,-99.156585,False
111,ASM,-70.294812,False
141,BCG,-81.396994,False
195,BMP,19.215057,True
223,BVH,12.881425,False
251,CII,171.206728,True
279,CMG,-29.380119,False
307,CTD,-1503.277407,False
363,CTR,9.635714,False


In [11]:
# Test the second condition
evaluate.check_eps_growth_2ndcondition()       

symbol
AAA    False
ANV    False
ASM    False
BCG    False
BMP     True
BVH    False
CII    False
CMG    False
CTD    False
CTR    False
DBC    False
DCM    False
DGC    False
DGW    False
DHC    False
DIG    False
DPM    False
DXG    False
FPT    False
GAS    False
GMD    False
HAG    False
HCM    False
HDC    False
HDG    False
HPG    False
HSG    False
HT1    False
IMP     True
KBC    False
KDC     True
KDH    False
MSN    False
MWG    False
NKG    False
NLG    False
NT2    False
PAN    False
PDR    False
PHR    False
PNJ    False
PPC    False
PTB    False
PVD    False
PVT    False
REE    False
SAM    False
SBT    False
SJS     True
SSI     True
TMS    False
VCG    False
VGC    False
VHC    False
VIC    False
VIX     True
VND    False
VNM    False
VSH    False
dtype: bool

In [12]:
# Test the third condition
evaluate.assess_eps_near_peak_3rdcondition()

symbol
AAA    False
ANV    False
ASM    False
BCG    False
BMP    False
BVH    False
CII    False
CMG    False
CTD     True
CTR     True
DBC    False
DCM    False
DGC    False
DGW     True
DHC    False
DIG    False
DPM    False
DXG    False
FPT    False
GAS    False
GMD    False
HAG    False
HCM     True
HDC    False
HDG    False
HPG     True
HSG    False
HT1    False
IMP    False
KBC    False
KDC    False
KDH    False
MSN    False
MWG    False
NKG    False
NLG    False
NT2    False
PAN     True
PDR    False
PHR    False
PNJ    False
PPC    False
PTB    False
PVD    False
PVT    False
REE    False
SAM    False
SBT    False
SJS    False
SSI     True
TMS    False
VCG     True
VGC    False
VHC    False
VIC    False
VIX    False
VND     True
VNM     True
VSH    False
dtype: bool

In [13]:
# Test the fourth condition
evaluate.check_revenue_growth_4thcondition()

/tmp/ipykernel_63131/77979599.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_quarter_df["condition_met"] = (


,symbol,revenue_growth_(%),condition_met
27,AAA,7.290048,False
83,ANV,-11.797869,False
111,ASM,-14.377499,False
141,BCG,-11.671474,False
195,BMP,-37.554390,False
223,BVH,-2.519686,False
251,CII,-65.543857,False
279,CMG,-1.726354,False
307,CTD,32.454055,True
363,CTR,18.344779,False


In [14]:
# Test the fifth condition
evaluate.check_accelerating_revenue_growth_5thcondition()

symbol
AAA    False
ANV    False
ASM    False
BCG    False
BMP    False
BVH    False
CII    False
CMG    False
CTD    False
CTR    False
DBC    False
DCM     True
DGC    False
DGW    False
DHC    False
DIG    False
DPM     True
DXG     True
FPT    False
GAS     True
GMD    False
HAG    False
HCM    False
HDC    False
HDG    False
HPG     True
HSG     True
HT1    False
IMP    False
KBC    False
KDC    False
KDH    False
MSN    False
MWG     True
NKG     True
NLG    False
NT2    False
PAN    False
PDR    False
PHR    False
PNJ    False
PPC    False
PTB    False
PVD    False
PVT    False
REE    False
SAM     True
SBT    False
SJS     True
SSI     True
TMS     True
VCG    False
VGC     True
VHC    False
VIC    False
VIX    False
VND     True
VNM    False
VSH    False
dtype: bool

In [15]:
# Test the sixth condition
evaluate.check_accelerating_profit_growth_6thcondition()

symbol
AAA    False
ANV    False
ASM    False
BCG    False
BMP    False
BVH    False
CII     True
CMG    False
CTD     True
CTR    False
DBC    False
DCM    False
DGC    False
DGW    False
DHC     True
DIG    False
DPM    False
DXG    False
FPT    False
GAS    False
GMD    False
HAG    False
HCM     True
HDC    False
HDG    False
HPG     True
HSG    False
HT1    False
IMP    False
KBC    False
KDC    False
KDH    False
MSN    False
MWG     True
NKG     True
NLG    False
NT2    False
PAN     True
PDR    False
PHR    False
PNJ    False
PPC    False
PTB     True
PVD     True
PVT    False
REE    False
SAM    False
SBT    False
SJS     True
SSI     True
TMS    False
VCG    False
VGC     True
VHC    False
VIC    False
VIX    False
VND     True
VNM     True
VSH    False
dtype: bool

In [16]:
# Test the seventh condition
evaluate.check_roe_7thcondition()

/tmp/ipykernel_63131/77979599.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current_quarter_df["condition_met"] = (


,symbol,roe_(%),condition_met
27,AAA,0.017459,False
83,ANV,0.050116,False
111,ASM,0.046025,False
141,BCG,-0.009286,False
195,BMP,0.367252,False
223,BVH,0.081056,False
251,CII,0.012075,False
279,CMG,0.122819,False
307,CTD,0.016666,False
363,CTR,0.289611,False


In [17]:
# Init technical
technical = TechnicalAnalysis(df_technical=df_technical_)

In [18]:
# Test the technical analysis
# Calculate EMA34-89 for each stock
technical.calculate_ema()

,symbol,date,open,high,low,close,volume,EMA34,EMA89
0,AAA,2017-01-03 00:00:00,12.50,12.95,12.50,12.81,694080,12.810000,12.810000
1,AAA,2017-01-04 00:00:00,13.13,13.23,12.76,12.97,740770,12.819143,12.813556
2,AAA,2017-01-05 00:00:00,13.02,13.23,12.42,13.23,1057020,12.842620,12.822810
3,AAA,2017-01-06 00:00:00,13.02,13.05,12.50,12.76,823010,12.837899,12.821414
4,AAA,2017-01-09 00:00:00,12.76,13.65,12.55,13.54,777440,12.878019,12.837383
...,...,...,...,...,...,...,...,...,...
163775,VSH,2023-12-28 00:00:00,44.80,46.00,44.50,45.00,12800,42.342256,41.491205
163776,VSH,2023-12-29 00:00:00,45.20,45.20,44.50,44.80,12900,42.482699,41.564734
163777,VSH,2024-01-02 00:00:00,44.80,45.30,44.60,45.30,14200,42.643687,41.647740
163778,VSH,2024-01-03 00:00:00,48.30,48.30,45.60,46.50,14300,42.864048,41.755568


In [19]:
#   Calculate MA5-10-20-50-100-200 for each stock
technical.calculate_ma()


,symbol,date,open,high,low,close,volume,EMA34,EMA89,MA5,MA10,MA20,MA50,MA100,MA200
0,AAA,2017-01-03 00:00:00,12.50,12.95,12.50,12.81,694080,12.810000,12.810000,NaN,NaN,NaN,NaN,NaN,NaN
1,AAA,2017-01-04 00:00:00,13.13,13.23,12.76,12.97,740770,12.819143,12.813556,NaN,NaN,NaN,NaN,NaN,NaN
2,AAA,2017-01-05 00:00:00,13.02,13.23,12.42,13.23,1057020,12.842620,12.822810,NaN,NaN,NaN,NaN,NaN,NaN
3,AAA,2017-01-06 00:00:00,13.02,13.05,12.50,12.76,823010,12.837899,12.821414,NaN,NaN,NaN,NaN,NaN,NaN
4,AAA,2017-01-09 00:00:00,12.76,13.65,12.55,13.54,777440,12.878019,12.837383,13.062,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163775,VSH,2023-12-28 00:00:00,44.80,46.00,44.50,45.00,12800,42.342256,41.491205,44.032,43.046,42.7130,41.6130,41.6589,39.72355
163776,VSH,2023-12-29 00:00:00,45.20,45.20,44.50,44.80,12900,42.482699,41.564734,44.314,43.365,42.8230,41.6740,41.6805,39.77775
163777,VSH,2024-01-02 00:00:00,44.80,45.30,44.60,45.30,14200,42.643687,41.647740,44.696,43.734,42.9560,41.7384,41.7015,39.83665
163778,VSH,2024-01-03 00:00:00,48.30,48.30,45.60,46.50,14300,42.864048,41.755568,45.280,44.223,43.1535,41.8362,41.7335,39.90200


In [20]:
# Check the 8th condition
technical.check_emacross_8thcondition()


symbol
AAA    True
ACB    True
AGG    True
ANV    True
ASM    True
       ... 
VNM    True
VPB    True
VPI    True
VRE    True
VSH    True
Name: EMA_Condition_Met, Length: 100, dtype: bool

In [1]:
from recommendation_service.stock_recommender import StockRecommender

# Init the class
stock_recommender = StockRecommender(
    latest_year="2023",
    latest_quarter="Q3 2023",
    recent_2_quarters=["Q3 2023", "Q2 2023"],
    recent_3_quarters=["Q1 2023", "Q2 2023", "Q3 2023"],
    growth_threshold_eps=15,
    growth_threshold_revenue=20,
    growth_threshold_roe=15,
)


In [2]:
recommended_stock = stock_recommender.get_recommendation()

2024-01-07 15:24:29,192 - stock_recommender.py:48        - INFO - Checking EPS growth for the most recent quarter
/home/tb24/projects/rule-based-stock-recommendation-system/recommendation_service/fundamentals_analysis.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_quarter_df["condition_1"] = (
2024-01-07 15:24:29,196 - stock_recommender.py:50        - INFO - Checking EPS growth for the two most recent quarters
2024-01-07 15:24:29,206 - stock_recommender.py:52        - INFO - Checking EPS growth for the last 12 months
2024-01-07 15:24:29,212 - stock_recommender.py:54        - INFO - Checking revenue growth for the most recent quarter
/home/tb24/projects/rule-based-stock-recommendation-system/recommendation_service/fundamentals_analysi

In [3]:
recommended_stock

,condition_1,condition_2,condition_3,condition_4,condition_5,condition_6,condition_7,EMA_Condition_Met,MA_Condition_Met,Total Conditions Met,Ranking
symbol,,,,,,,,,,,
SSI,True,True,True,True,True,True,False,True,True,8,1
SJS,True,True,False,True,True,True,False,True,True,7,2
VND,True,False,True,True,True,True,False,True,True,7,3
HPG,False,False,True,False,True,True,False,True,True,5,4
IMP,True,True,False,True,False,False,False,True,True,5,5
...,...,...,...,...,...,...,...,...,...,...,...
BCM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,2,96
BID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,2,97
BWE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,2,98


In [ ]:
stock_recommender.technical

# Portfolio Optimizer

In [50]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datetime import datetime, timedelta
from utils.timescale_connector import TimescaleConnector
from recommendation_service.stock_recommender import StockRecommender

# Init the class
stock_recommender = StockRecommender(
    latest_year="2023",
    latest_quarter="Q3 2023",
    recent_2_quarters=["Q3 2023", "Q2 2023"],
    recent_3_quarters=["Q1 2023", "Q2 2023", "Q3 2023"],
    growth_threshold_eps=15,
    growth_threshold_revenue=20,
    growth_threshold_roe=15,
)
recommended_stock = stock_recommender.get_recommendation()

2024-01-07 20:58:59,157 - stock_recommender.py:48        - INFO - Checking EPS growth for the most recent quarter
/home/tb24/projects/rule-based-stock-recommendation-system/recommendation_service/fundamentals_analysis.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_quarter_df["condition_1"] = (
2024-01-07 20:58:59,159 - stock_recommender.py:50        - INFO - Checking EPS growth for the two most recent quarters
2024-01-07 20:58:59,165 - stock_recommender.py:52        - INFO - Checking EPS growth for the last 12 months
2024-01-07 20:58:59,168 - stock_recommender.py:54        - INFO - Checking revenue growth for the most recent quarter
/home/tb24/projects/rule-based-stock-recommendation-system/recommendation_service/fundamentals_analysi

In [51]:
top_3 = recommended_stock.head(3).index.tolist()

In [52]:
df = TimescaleConnector.query_ohlcv_daily("SSI")
df

,symbol,open,high,low,close,volume
date,,,,,,
2017-01-03,SSI,8.28,8.51,8.26,8.47,1300330
2017-01-04,SSI,8.47,8.55,8.41,8.47,1224340
2017-01-05,SSI,8.49,8.53,8.45,8.49,565830
2017-01-06,SSI,8.49,8.77,8.49,8.58,2995300
2017-01-09,SSI,8.58,8.70,8.58,8.66,886970
...,...,...,...,...,...,...
2023-12-28,SSI,32.85,33.15,32.65,33.00,10895900
2023-12-29,SSI,32.95,33.10,32.75,32.80,12790300
2024-01-02,SSI,33.00,33.20,32.50,32.50,18052300


In [53]:
# Query the prices for the top 3 stocks
for symbol in top_3:
    df = TimescaleConnector.query_ohlcv_daily(symbol)
    # Get the close price, and concat them into a dataframe, with the date, and symbol
    df = df[["close"]]
    df.columns = [symbol]
    if symbol == top_3[0]:
        top_stocks_df = df
    else:
        top_stocks_df = pd.concat([top_stocks_df, df], axis=1)  

In [54]:
# Calculate the Log Returns
# Change dtypes to float
top_stocks_df = top_stocks_df.astype(float)
log_returns = np.log(top_stocks_df / top_stocks_df.shift(1)).dropna()

In [55]:
# Calculate the Covariance Matrix and Annualize it
cov_matrix_annual = log_returns.cov() * 252
cov_matrix_annual

# Calculate the porfolio standard deviation
def standard_deviation (weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)
# Calculate the expected return 
def expected_return (weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252
# Calculate the sharpe ratio
def sharpe_ratio (weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return (weights, log_returns) - risk_free_rate) / standard_deviation (weights, cov_matrix) 
# Set the risk free rate 
risk_free_rate = 0.02
#-----------------------------
def neg_sharpe_ratio (weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

In [56]:
# Set the constraints and bounds
constraints = {'type' : 'eq', 'fun': lambda weights: np.sum(weights) - 1}
bounds = [(0, 0.5) for _ in range(len(top_3))]
# Set the initial weights
initial_weights = np.array([1/len(top_3)]*len(top_3))
# Optimize the weights to maximize the Sharpe Ratio
optimized_results = minimize(neg_sharpe_ratio, initial_weights,args=(log_returns, cov_matrix_annual, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)
# Get optimal weights 
optimal_weights = optimized_results.x
print("Optimal Weights:")
for ticker, weight in zip(top_3, optimal_weights):
    print(f"{ticker}: {weight:.4f}")

optimal_portfolio_return = expected_return(optimal_weights, log_returns)
optimal_portfolio_volatility = standard_deviation(optimal_weights, cov_matrix_annual)
optimal_sharpe_ratio = sharpe_ratio(optimal_weights, log_returns, cov_matrix_annual, risk_free_rate)

print(f"Expected Annual Return: {optimal_portfolio_return:.4f}")
print(f"Expected Volatility: {optimal_portfolio_volatility:.4f}")
print(f"Sharpe Ratio: {optimal_sharpe_ratio:.4f}")

Optimal Weights:
SSI: 0.0000
SJS: 0.5000
VND: 0.5000
Expected Annual Return: 0.2514
Expected Volatility: 0.3287
Sharpe Ratio: 0.7040


# Re-do Industries Ranking

In [79]:
import pandas as pd

df = pd.read_csv("data/industry_sector_analysis.csv", encoding="ISO-8859-1")

In [80]:
symbol_industry_df = df[["symbol", "industry sector"]].drop_duplicates()
symbol_industry_df.rename(
    columns={"industry sector": "industry_sector"}, inplace=True
)

In [81]:
# Correcting the industry sector names

corrections = {
    "Hóa Ch?t": "Hóa Chất",
    "Hóa ch?t": "Hóa Chất",
    "B?t ??ng s?n": "Bất Động Sản",
    "S?n xu?t th?c ph?m": "Sản xuất thực phẩm",
    "D?ch v? tài chính": "Dịch vụ tài chính",
    "Xây d?ng và v?t li?u": "Xây dựng và vật liệu",
    "B?o hi?m nhân th?": "Bảo hiểm nhân thọ",
    "N??c và khí ??t": "Nước và khí đốt",
    "Ph?n m?m d?ch v? máy tính": "Phần mềm dịch vụ máy tính",
    "Bán l?": "Bán lẻ",
    "Lâm nghi?p và gi?y": "Lâm nghiệp và giấy",
    "Ph?n m?m và d?ch v? máy tính": "Phần mềm và dịch vụ máy tính",
    "S?n xu?t và phân ph?i ?i?n": "Sản xuất và phân phối điện",
    "?i?n t? và thi?t b? ?i?n": "Điện tử và thiết bị điện",
    "V?n t?i": "Vận tải",
    "Kim lo?i": "Kim loại",
    "D??c ph?m": "Dược phẩm",
    "S?n xu?t d??c ph?m": "Sản xuất dược phẩm",
    "S?n xu?t d?u khí": "Sản xuất dầu khí",
    "Hàng cá nhân": "Hàng cá nhân",
    "Thi?t b?, d?ch v? và phân ph?i d?u khí": "Thiết bị, dịch vụ và phân phối dầu khí",
    "Công nghi?p n?ng": "Công nghiệp nặng",
    "Bia và ?? u?ng": "Bia và đồ uống",
    "Thi?t b? và ph?n c?ng": "Thiết bị và phần cứng",
    "Du l?ch và gi?i trí": "Du lịch và giải trí",
    "Ch? s? th? tr??ng chung": "Chỉ số thị trường chung",
}


In [82]:
symbol_industry_df.industry_sector = symbol_industry_df.industry_sector.replace(
    corrections
)

In [83]:
symbol_industry_df

,symbol,industry_sector
0,AAA,Hóa Chất
250,ACB,Ngân hàng
500,AGG,Bất Động Sản
750,ANV,Sản xuất thực phẩm
1000,ASM,Sản xuất thực phẩm
...,...,...
23987,VPB,Ngân hàng
24237,VPI,Bất Động Sản
24487,VRE,Bất Động Sản
24737,VSH,Sản xuất và phân phối điện


In [84]:
# Ranking
from datetime import datetime, timedelta

prices_1y = TimescaleConnector.query_ohlcv_1y_interval()

In [85]:
# Merge the industry sector with the prices
prices_1y = prices_1y.merge(symbol_industry_df, on="symbol", how="left")

In [86]:
prices_1y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24688 entries, 0 to 24687
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   symbol           24688 non-null  object
 1   date             24688 non-null  object
 2   open             24688 non-null  object
 3   high             24688 non-null  object
 4   low              24688 non-null  object
 5   close            24688 non-null  object
 6   volume           24688 non-null  object
 7   industry_sector  24688 non-null  object
dtypes: object(8)
memory usage: 1.5+ MB


In [87]:
vnindex_1y = TimescaleConnector.query_vnindex_1y_interval()

In [88]:
vnindex_1y

,date,open,high,low,close,volume,percent_change
0,2023-01-10T00:00:00,1054.84,1057.44,1045.75,1053.35,5.462631e+08,-0.081644
1,2023-01-11T00:00:00,1058.25,1065.69,1054.53,1055.76,5.451844e+08,0.228272
2,2023-01-12T00:00:00,1058.51,1060.97,1050.27,1056.39,5.064669e+08,0.059637
3,2023-01-13T00:00:00,1060.71,1066.39,1055.23,1060.17,6.702317e+08,0.356547
4,2023-01-16T00:00:00,1062.27,1067.74,1057.87,1066.68,5.207909e+08,0.610305
...,...,...,...,...,...,...,...
242,2023-12-28T00:00:00,1122.77,1130.58,1120.50,1128.93,7.169279e+08,0.614741
243,2023-12-29T00:00:00,1131.75,1135.05,1128.98,1129.93,7.108476e+08,0.088501
244,2024-01-02T00:00:00,1136.39,1139.71,1128.69,1131.72,8.422555e+08,0.158166
245,2024-01-03T00:00:00,1131.97,1144.17,1128.32,1144.17,7.199327e+08,1.088125


In [89]:
# Calculate percentage change
prices_1y = prices_1y.astype({"close": float})  


In [90]:
# prices_1y["industry_percent_change"] = prices_1y.groupby("symbol")["close"].pct_change()

prices_1y["stock_price_change"] = prices_1y.groupby(
    "symbol"
).close.pct_change()

In [91]:
prices_1y

,symbol,date,open,high,low,close,volume,industry_sector,stock_price_change
0,AAA,2023-01-10 00:00:00,7.00,7.17,6.97,7.07,1397400,Hóa Chất,NaN
1,AAA,2023-01-11 00:00:00,7.15,7.25,7.10,7.19,1038300,Hóa Chất,0.016973
2,AAA,2023-01-12 00:00:00,7.19,7.25,7.15,7.19,850100,Hóa Chất,0.000000
3,AAA,2023-01-13 00:00:00,7.25,7.28,7.12,7.12,1754700,Hóa Chất,-0.009736
4,AAA,2023-01-16 00:00:00,7.15,7.19,7.10,7.16,1240600,Hóa Chất,0.005618
...,...,...,...,...,...,...,...,...,...
24683,VSH,2023-12-28 00:00:00,44.80,46.00,44.50,45.00,12800,Sản xuất và phân phối điện,0.004464
24684,VSH,2023-12-29 00:00:00,45.20,45.20,44.50,44.80,12900,Sản xuất và phân phối điện,-0.004444
24685,VSH,2024-01-02 00:00:00,44.80,45.30,44.60,45.30,14200,Sản xuất và phân phối điện,0.011161
24686,VSH,2024-01-03 00:00:00,48.30,48.30,45.60,46.50,14300,Sản xuất và phân phối điện,0.026490


In [92]:
# Merged with the vnindex
vnindex_1y.dropna()
prices_1y.dropna()

vnindex_1y.date = pd.to_datetime(vnindex_1y.date)
prices_1y.date = pd.to_datetime(prices_1y.date)

prices_merged = prices_1y.merge(vnindex_1y[['date', 'percent_change']], on='date', how='left').dropna()
prices_merged

,symbol,date,open,high,low,close,volume,industry_sector,stock_price_change,percent_change
1,AAA,2023-01-11,7.15,7.25,7.10,7.19,1038300,Hóa Chất,0.016973,0.228272
2,AAA,2023-01-12,7.19,7.25,7.15,7.19,850100,Hóa Chất,0.000000,0.059637
3,AAA,2023-01-13,7.25,7.28,7.12,7.12,1754700,Hóa Chất,-0.009736,0.356547
4,AAA,2023-01-16,7.15,7.19,7.10,7.16,1240600,Hóa Chất,0.005618,0.610305
5,AAA,2023-01-17,7.24,7.40,7.17,7.40,2365400,Hóa Chất,0.033520,1.985684
...,...,...,...,...,...,...,...,...,...,...
24683,VSH,2023-12-28,44.80,46.00,44.50,45.00,12800,Sản xuất và phân phối điện,0.004464,0.614741
24684,VSH,2023-12-29,45.20,45.20,44.50,44.80,12900,Sản xuất và phân phối điện,-0.004444,0.088501
24685,VSH,2024-01-02,44.80,45.30,44.60,45.30,14200,Sản xuất và phân phối điện,0.011161,0.158166
24686,VSH,2024-01-03,48.30,48.30,45.60,46.50,14300,Sản xuất và phân phối điện,0.026490,1.088125


In [93]:
prices_merged

,symbol,date,open,high,low,close,volume,industry_sector,stock_price_change,percent_change
1,AAA,2023-01-11,7.15,7.25,7.10,7.19,1038300,Hóa Chất,0.016973,0.228272
2,AAA,2023-01-12,7.19,7.25,7.15,7.19,850100,Hóa Chất,0.000000,0.059637
3,AAA,2023-01-13,7.25,7.28,7.12,7.12,1754700,Hóa Chất,-0.009736,0.356547
4,AAA,2023-01-16,7.15,7.19,7.10,7.16,1240600,Hóa Chất,0.005618,0.610305
5,AAA,2023-01-17,7.24,7.40,7.17,7.40,2365400,Hóa Chất,0.033520,1.985684
...,...,...,...,...,...,...,...,...,...,...
24683,VSH,2023-12-28,44.80,46.00,44.50,45.00,12800,Sản xuất và phân phối điện,0.004464,0.614741
24684,VSH,2023-12-29,45.20,45.20,44.50,44.80,12900,Sản xuất và phân phối điện,-0.004444,0.088501
24685,VSH,2024-01-02,44.80,45.30,44.60,45.30,14200,Sản xuất và phân phối điện,0.011161,0.158166
24686,VSH,2024-01-03,48.30,48.30,45.60,46.50,14300,Sản xuất và phân phối điện,0.026490,1.088125


In [94]:
grouped = prices_merged.groupby("industry_sector").agg(
    {"stock_price_change": "mean", "percent_change": "mean"}
)

In [95]:
grouped

,stock_price_change,percent_change
industry_sector,,
Bia và đồ uống,-0.001112,0.030347
Bán lẻ,0.001592,0.030347
Bảo hiểm nhân thọ,-0.000566,0.030347
Bất Động Sản,0.001103,0.030347
Công nghiệp nặng,-0.000177,0.030347
Du lịch và giải trí,0.000025,0.030347
Dược phẩm,0.000745,0.035924
Dịch vụ tài chính,0.002899,0.030347
Hàng cá nhân,0.000012,0.030347


In [97]:
grouped["RS"] = grouped["stock_price_change"] / grouped["percent_change"]
ranked_sectors = grouped.sort_values(by="RS", ascending=False)
ranked_sectors.reset_index(inplace=True)
ranked_sectors["ranking"] = ranked_sectors.reset_index(drop=False).index + 1

In [100]:
ranked_sectors.industry_sector.unique().tolist()

['Dịch vụ tài chính',
 'Điện tử và thiết bị điện',
 'Kim loại',
 'Xây dựng và vật liệu',
 'Thiết bị, dịch vụ và phân phối dầu khí',
 'Bán lẻ',
 'Phần mềm và dịch vụ máy tính',
 'Hóa Chất',
 'Phần mềm dịch vụ máy tính',
 'Vận tải',
 'Lâm nghiệp và giấy',
 'Bất Động Sản',
 'Sản xuất thực phẩm',
 'Ngân hàng',
 'Dược phẩm',
 'Thiết bị và phần cứng',
 'Sản xuất và phân phối điện',
 'Sản xuất dược phẩm',
 'Du lịch và giải trí',
 'Sản xuất dầu khí',
 'Hàng cá nhân',
 'Công nghiệp nặng',
 'Nước và khí đốt',
 'Bảo hiểm nhân thọ',
 'Bia và đồ uống']

In [1]:
from utils.timescale_connector import TimescaleConnnector2
import pandas as pd

df = TimescaleConnnector2.query_update_price(symbol="SSI")

change = TimescaleConnnector2.query_update_change(symbol="SSI").values

# Round the values to 2 decimal places
# df = df.round(2)
# change = round(change, 2)


processed_df = pd.DataFrame(
    {
        "code": df["symbol"],
        "open": df["open"],
        "high": df["high"],
        "low": df["low"],
        "close": df["close"],
        "volume": df["vol"],
        "trend": 'up' if change >= 0 else 'down',
    }
)

In [2]:
processed_df.to_dict(orient="records")

[{'code': 'SSI',
  'open': 12215.55,
  'high': 12215.55,
  'low': 12215.55,
  'close': 12215.55,
  'volume': 52132.619634525836,
  'trend': 'down'}]